<a href="https://www.kaggle.com/code/annamalkova88/psychopathy-prediction-my-vers?scriptVersionId=130094300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The model should aim to identify people scoring high in Psychopathy, for the purpose of this competition, defined as 2 SD's above a mean of 1.98. This accounts for roughly 3% of the entire sample and therefore the challenge with this dataset is developing a model to work with a highly imbalanced dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif
import plotly 
import plotly.express as px

# Data info

In [ ]:
df = pd.read_csv('/kaggle/input/twitter-psychopathy-prediction/PersonalityData_ExternalVersion001.csv')

In [ ]:
df.head()

In [ ]:
df['psycho_status'] = df['psychopathy'].apply(lambda x: 1 if x>1.98 else 0)

In [ ]:
df.describe()

In [ ]:
temp = df['psycho_status'].value_counts(normalize = True)
temp

In [ ]:
fig = px.histogram(data_frame = df,
                   x = 'psycho_status', 
                   color = 'psycho_status'
                  )
fig.show()

In [ ]:
mask = list(df.loc[:, 'openness':'neuroticism'].columns)
fig = px.box(data_frame = df,
                   x = mask
                  )
fig.show()

In [ ]:
mask = list(df.loc[:, 'psychopathy':'narcissism'].columns)
fig = px.box(data_frame = df,
                   x = mask
                  )
fig.show()

In [ ]:
mask = list(df.loc[:, 'psychopathy':'neuroticism'].columns)
fig = px.box(data_frame = df,
                   x = mask,
             color = 'psycho_status'
                  )
fig.show()

In [ ]:
mask = list(df.loc[:, 'machiavellianism':'neuroticism'].columns)
df.corr()['psycho_status'][mask].apply(lambda x: abs(x)).sort_values().plot.barh()
plt.show

In [ ]:
mask = list(df.loc[:, 'psychopathy':'neuroticism'].columns)
plt.figure(figsize=(20,20))  
sns.heatmap(df[mask].corr(),fmt='.2f',annot=True, center=0)

# EDA

Work with NaN

In [ ]:
a = df.isnull().mean()*100
b = a[a>0].sort_values(ascending = False)
b[b>1]

In [ ]:
mask = list(df.loc[:, 'Var1':'Var100'].columns)

In [ ]:
for el in mask:
    df[el] = df[el].fillna(df[el].mode().iloc[0])

In [ ]:
a = df.isnull().mean()*100
b = a[a>0].sort_values(ascending = False)
b

In [ ]:
#check for categorical
types = df.dtypes
cat_features = list(types[(types == 'object')].index)
cat_features

In [ ]:
df['Var20'] = df['Var20'].apply(lambda x: 1 if x=='1' else 0)

In [ ]:
df = df.drop(['uid'],axis = 1)

In [ ]:
df_class = df.drop(['psychopathy'],axis = 1)

# Classification

In [ ]:
X = df_class.drop(['psycho_status'], axis=1)
y = df_class['psycho_status']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)
print(X_train.shape, 
      X_test.shape)
print(y_train.value_counts(normalize = True), 
      y_test.value_counts(normalize = True))

In [ ]:
#normalization
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns= X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
# Logistic regression
log_reg = linear_model.LogisticRegression(random_state=42, solver = 'sag', max_iter=1000)

log_reg.fit(X_train_scaled, y_train)

y_pred_train = log_reg.predict(X_train_scaled)
y_pred_test = log_reg.predict(X_test_scaled)
print(metrics.classification_report(y_train, y_pred_train))
print(metrics.classification_report(y_test, y_pred_test))

In [ ]:
# Decision Tree
dt_clf = tree.DecisionTreeClassifier(
    criterion='entropy', 
    random_state=42,
    max_depth=11
)

dt_clf.fit(X_train_scaled, y_train)
y_pred_train = dt_clf.predict(X_train_scaled)
y_pred_test = dt_clf.predict(X_test_scaled)
print(metrics.classification_report(y_train, y_pred_train))
print(metrics.classification_report(y_test, y_pred_test))

# Ensambles

In [ ]:
# Random forest
rf_clf_full = ensemble.RandomForestClassifier(
    n_estimators=100, 
    criterion='gini',
    max_depth=10, 
    max_features='sqrt', 
    random_state=42, 
    min_samples_leaf = 5
)

rf_clf_full.fit(X_train_scaled, y_train)


y_pred_test = rf_clf_full.predict(X_test_scaled)

print(metrics.classification_report(y_test, y_pred_test))

In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

gb = GradientBoostingClassifier(
    learning_rate=0.05, 
    n_estimators=300, 
    max_depth=5, 
    random_state=42, 
    min_samples_leaf = 5
)

gb.fit(X_train_scaled, y_train)

y_pred_test = gb.predict(X_test_scaled)

print(classification_report(y_test, y_pred_test))

In [ ]:
# Optuna optimization
import optuna
random_state = 42
def optuna_rf(trial):
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  model.fit(X_train_scaled, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train_scaled))

  return score

study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study.optimize(optuna_rf, n_trials=20)

print("The best hyperparameters {}".format(study.best_params))
print("f1_score for train sample: {:.2f}".format(study.best_value))

In [ ]:
# metrics for test sample
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
print("accuracy for test samples: {:.2f}".format(model.score(X_test_scaled, y_test)))
y_test_pred = model.predict(X_test_scaled)
print('f1_score for test samples: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

# Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

estimators = [
    ('dt',  DecisionTreeClassifier(
        criterion='entropy', 
        random_state=42,
        max_depth=10)),
    ('rf_opt', RandomForestClassifier(
        **study.best_params,
        random_state=42)),
    ('log', LogisticRegression(
        random_state=42,
        solver = 'sag', 
        max_iter=1000
    ))
]


reg = ensemble.StackingClassifier(
    estimators=estimators,
    final_estimator=linear_model.LogisticRegression(
                                          random_state=42)
)

reg.fit(X_train_scaled, y_train)

In [ ]:
y_pred_stack = reg.predict(X_test_scaled)

print(classification_report(y_test, y_pred_stack))

In [ ]:
# evaluate which features show the greatest importance in the gradient boosting model
fig, ax = plt.subplots(figsize=(20, 5)) 
feature = X_train_scaled.columns 
feature_importances = gb.feature_importances_ 

sns.barplot(x=feature, y=feature_importances, ax=ax);

ax.set_title('Bar plot feature importances')
ax.set_xlabel('Features')
ax.set_ylabel('Importances');

# Decrease number of features

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
reg.fit(X_train_pca, y_train)
y_pred_stack = reg.predict(X_test_pca)

print(classification_report(y_test, y_pred_stack))

#Conclusions

Logistic regression
f1 score = 0.69
accuracy = 0.71

Decision Tree
f1 score = 0.69
accuracy = 0.71

Random Forest
f1 score = 0.65
accuracy = 0.68
after optuna
f1 score = 0.64
accuracy = 0.68

Gradient boosting
f1 score = 0.66
accuracy = 0.69

Stacking
f1 score = 0.69
accuracy = 0.71

PCA+stacking
f1 score = 0.70
accuracy = 0.72


# Clusterization

In [ ]:
from sklearn.cluster import KMeans 
k_clust = 2
kmean= KMeans(k_clust)
kmean.fit(X) 
labels=kmean.labels_

In [ ]:
clusters=pd.concat([df_class, pd.DataFrame({'cluster':labels})], axis=1)
clusters.head()

In [ ]:
y_done = clusters['psycho_status']
y_cluster = clusters['cluster']
print(metrics.classification_report(y_done, y_cluster))

# Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor

import xgboost as xgb

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_classif
# Importing DataScience libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from scipy import stats
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler,LabelEncoder,OrdinalEncoder,power_transform
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
import warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
df_regr = df.drop('psycho_status', axis = 1)
X = df_regr.drop('psychopathy', axis = 1)
y = df_regr['psychopathy']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)
print(X_train.shape, 
      X_test.shape)


In [ ]:
#normalization
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns= X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

### Multiple Regression

In [ ]:
linear = linear_model.LinearRegression().fit(X_train_scaled, y_train)
y_train_predict = linear.predict(X_train_scaled)
y_test_predict = linear.predict(X_test_scaled)
print("Multiple:", metrics.r2_score(y_train, y_train_predict))
print("Multiple:", metrics.r2_score(y_test, y_test_predict))

In [ ]:
import optuna
def objective(trial):
    # Define the hyperparameters to optimize
    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)

    # Create an instance of the linear regression model with the hyperparameters
    model = Ridge(alpha=alpha)

    # Train the model
    model.fit(X_train_scaled, y_train)
    score = metrics.r2_score(y_train, model.predict(X_train_scaled))
    return score

In [ ]:
study = optuna.create_study(study_name="LinearRegression", direction="maximize")

study.optimize(objective, n_trials=30)

In [ ]:
print("Best hyperparameters {}".format(study.best_params))
print("r2_score train sample: {:.2f}".format(study.best_value))

In [ ]:
model = linear_model.Ridge(**study.best_params,random_state=42)
model.fit(X_train_scaled, y_train)

y_test_pred = model.predict(X_test_scaled)
print('r2_score train sample: {:.2f}'.format(metrics.r2_score(y_test, y_test_pred)))

### Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = preprocessing.PolynomialFeatures(degree=3, include_bias=False)
poly.fit(X_train_scaled)
X_train_poly = poly.transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)
model = linear_model.LinearRegression()
model.fit(X_train_poly, y_train)
y_train_predict = model.predict(X_train_poly)
y_test_predict = model.predict(X_test_poly)
print("poly:", metrics.r2_score(y_train, y_train_predict))
print("poly:", metrics.r2_score(y_test, y_test_predict))

In [ ]:
lasso_lr_poly = linear_model.Lasso(alpha=0.1)

lasso_lr_poly.fit(X_train_poly, y_train)

y_train_predict_poly = lasso_lr_poly.predict(X_train_poly)

y_test_predict_poly = lasso_lr_poly.predict(X_test_poly)

print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict_poly)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict_poly)))

### Ridge Regression

In [ ]:
ridge = linear_model.Ridge(alpha=300, fit_intercept=True, max_iter=20000, solver='saga').fit(X_train_scaled, y_train)
y_train_predict = ridge.predict(X_train_scaled)
y_test_predict = ridge.predict(X_test_scaled)
print("ridge:", metrics.r2_score(y_train, y_train_predict))
print("ridge:", metrics.r2_score(y_test, y_test_predict))

### LASSO Regression

In [ ]:
lasso = linear_model.Lasso(alpha =0.0005, random_state=1).fit(X_train, y_train)
y_predict = ridge.predict(X_test)
print("Lasso:", metrics.r2_score(y_test, y_predict))